<a href="https://colab.research.google.com/github/RhysComissiong/Python-Machine-Learning-Tutorial/blob/master/GANs_ch_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8 MB 40 kB/s 
     |████████████████████████████████| 3.8 MB 37.4 MB/s 
     |████████████████████████████████| 449 kB 39.8 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires tensorboard~=2.6, but you have tensorboard 2.0.2 which is incompatible.
tensorflow 2.7.0 requires tensorflow-estimator<2.8,~=2.7.0rc0, but you have tensorflow-estimator 2.0.1 which is incompatible.
tensorflow-probability 0.15.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [3]:
print("GPU Available:", tf.test.is_gpu_available())

GPU Available: True


In [4]:
if tf.test.is_gpu_available():
    device_name = tf.test.gpu_device_name()

else:
    device_name = '/CPU:0'

In [5]:
print(device_name)

/device:GPU:0


In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive
